# Homework 2

## Interior point method implementation

In [3]:
import numpy as np

### 4)

In [4]:
def next_mu(mu, m):
    return (1.0 - 1.0/(6.0*np.sqrt(m))) * mu


def pd_short_step_update(A, b, c, x, y, s, mu, safeguard=False, tau=0.99):
    A = np.asarray(A, dtype=float)
    b = np.asarray(b, dtype=float)
    c = np.asarray(c, dtype=float)
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    s = np.asarray(s, dtype=float)

    m, n = A.shape
    mup = next_mu(mu, m)

    r = mup - x * s  

    # Build M = A * diag(x/s) * A^T
    D = x / s
    M = A @ (D[:, None] * A.T)  # (m,m)

    # rhs = - A * (r/s)
    rhs = - A @ (r / s)

    # Solve for Δy
    Dy = np.linalg.solve(M, rhs)

    # Δs and Δx
    Ds = - A.T @ Dy
    Dx = (r + x * Ds) / s

    # Step length 
    alpha = 1.0
    if safeguard:

        idx_neg_dx = Dx < 0
        idx_neg_ds = Ds < 0
        if np.any(idx_neg_dx):
            alpha = min(alpha, tau * np.min(-x[idx_neg_dx] / Dx[idx_neg_dx]))
        if np.any(idx_neg_ds):
            alpha = min(alpha, tau * np.min(-s[idx_neg_ds] / Ds[idx_neg_ds]))
        alpha = float(alpha) if alpha > 0 else 1.0  

    xp = x + alpha * Dx
    yp = y + alpha * Dy
    sp = s + alpha * Ds
    mup = mup  

    return xp, yp, sp, mup

### 6) 

In [6]:
A = np.array([
    [3, 3, 3, 0, 0],
    [3, 1, 0, 1, 0],
    [1, 4, 0, 0, 1]
], dtype=float)

b = np.array([4, 3, 4], dtype=float)

c = np.array([-3, -4, 0, 0, 0], dtype=float)
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15], dtype=float)
y = np.array([-4/5, -4/5, -2/3], dtype=float)
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3], dtype=float)

# Checks
print("Ax = b ?", np.allclose(A @ x, b))
print("A^T y + s = c ?", np.allclose(A.T @ y + s, c))
print("x > 0 ?", np.all(x > 0))
print("s > 0 ?", np.all(s > 0))



Ax = b ? True
A^T y + s = c ? True
x > 0 ? True
s > 0 ? True


### 7)

### 8)

In [7]:
pd_short_step_update(A, b, c, x, y, s, 1)

(array([0.32503257, 0.42457191, 0.3713288 , 1.0264696 , 1.29411642]),
 array([-0.78951181, -0.73479005, -0.6386912 ]),
 array([2.21159678, 1.6580903 , 2.36853543, 0.73479005, 0.6386912 ]),
 np.float64(0.9037749551350623))